# Fulcra Total Recall Notebook
### Use this example to recall what you did on an arbitrary day by using date and requesting data

Dated: 25-Oct-2023

### Setting up FulcraAPI and Installing required libraries

In [ ]:
%pip install -qU fulcra-api
%pip install -qU folium
%pip install -qU matplotlib

Colaboratory note: You may see a dependency error while installing pyarrow; this can be ignored.

### Importing libraries we need

In [ ]:
from datetime import datetime
from collections import Counter
from IPython.display import display, HTML
from fulcra_api.core import FulcraAPI
import folium
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import concurrent.futures

### Before using Fulcra API

In [ ]:
fulcra = FulcraAPI()
fulcra.authorize()

### Recalling a day
Please enter a date and run the next cell, make sure you are authorized to access Fulcra backend

In [ ]:
recall_date = "25-10-2023"

### Initiate a Recall for your date

In [ ]:
metrics_ = ["AFibBurden","ActiveCaloriesBurned","AlcoholicDrinksConsumed","AppetiteChange","AppleWatchExerciseTime","AppleWatchMoveTime","AppleWatchStandTime","BasalBodyTemperature","BasalCaloriesBurned","BloodAlcoholContent","BloodGlucose","BloodOxygenSaturation","BloodPressureDiastolic","BloodPressureSystolic","BodyFatPercentage","BodyMassIndex","BodyTemperature","CaloriesConsumed","CervicalMucusQuality","ContraceptiveUse","DietaryBiotinConsumed","DietaryCaffeineConsumed","DietaryCalciumConsumed","DietaryCarbohydratesConsumed","DietaryChlorideConsumed","DietaryCholesterolConsumed","DietaryChromiumConsumed","DietaryCopperConsumed","DietaryFiberConsumed","DietaryFolateConsumed","DietaryIodineConsumed","DietaryIronConsumed","DietaryMagnesiumConsumed","DietaryManganeseConsumed","DietaryMolybdenumConsumed","DietaryNiacinConsumed","DietaryPantothenicAcidConsumed","DietaryPhosphorusConsumed","DietaryPotassiumConsumed","DietaryProteinConsumed","DietaryRiboflavinConsumed","DietarySeleniumConsumed","DietarySodiumConsumed","DietarySugarConsumed","DietaryThiaminConsumed","DietaryVitaminAConsumed","DietaryVitaminB12Consumed","DietaryVitaminB6Consumed","DietaryVitaminCConsumed","DietaryVitaminDConsumed","DietaryVitaminEConsumed","DietaryVitaminKConsumed","DietaryWaterConsumed","DietaryZincConsumed","DistanceTraveledCycling","DistanceTraveledDownhillSnowSports","DistanceTraveledOnFoot","DistanceTraveledSwimming","DistanceTraveledWithWheelchair","EnvironmentalAudioLevel","EnvironmentalAudioLevelIncraeseEvent","FallCount","HandwashingEvent","HeadphonesAudioLevel","HeadphonesAudioLevelIncraeseEvent","HeartRate","HeartRateRecoveryOneMinute","HeartRateVariabilitySDNN","Height","HighHeartRateEvent","InfrequentMenstrualCycles","InhalerUse","InsulinUnitsDelivered","IrregularHeartRhythmEvent","IrregularMenstrualCycles","Lactation","LeanBodyMass","LowCardioFitnessEvent","LowHeartRateEvent","MenstrualFlow","MindfulSession","MonounsaturatedFatConsumed","MoodChanges","NikeFuelPoints","OvulationTestResult","PersistentMenstrualBleeding","PolyunsaturatedFatConsumed","Pregnancy","PregnancyTestResult","ProgesteroneTestResult","ProlongedMenstrualPeriods","ReportedMood","RespiratoryRate","RestingHeartRate","RunningGroundContactTime","RunningPower","RunningSpeed","RunningStrideLength","RunningVerticalOscillation","SaturatedFatConsumed","SexualActivity","SixMinuteWalkDistance","SleepChanges","SleepStage","SleepingWristTemperature","StairAscentSpeed","StairDescentSpeed","StairFlightsClimbed","StandHour","StepCount","SwimmingStrokeCount","SymptomAbdominalCramps","SymptomAcne","SymptomBladderIncontinence","SymptomBloating","SymptomBreastPain","SymptomChestTightnessOrPain","SymptomChills","SymptomConstipation","SymptomCoughing","SymptomDiarrhea","SymptomDizziness","SymptomDrySkin","SymptomFainting","SymptomFatigue","SymptomFever","SymptomGeneralizedBodyAche","SymptomHairLoss","SymptomHeadache","SymptomHeartburn","SymptomHotFlashes","SymptomLossOfSmell","SymptomLossOfTaste","SymptomLowerBackPain","SymptomMemoryLapse","SymptomNausea","SymptomNightSweats","SymptomPelvicPain","SymptomRapidPoundingOrFlutteringHeartbeat","SymptomRunnyNose","SymptomShortnessOfBreath","SymptomSinusCongestion","SymptomSkippedHeartbeat","SymptomSoreThroat","SymptomVaginalDryness","SymptomVomiting","SymptomWheezing","ToothbrushingEvent","TotalFatConsumed","UnderwaterDepth","VO2Max","WaistCircumference","WalkingAsymmetry","WalkingDoubleSupport","WalkingHeartRate","WalkingSpeed","WalkingSteadiness","WalkingSteadinessDecreaseEvent","WalkingStrideLength","WaterTemperature","Weight","WheelchairPushes"]

recall_date_list = [int(num) for num in recall_date.split("-")]
s_year = recall_date_list[2]
s_month = recall_date_list[1]
s_day = recall_date_list[0]

# End date
e_year = recall_date_list[2]
e_month = recall_date_list[1]
e_day = recall_date_list[0] + 1

# Creating a datetime object
start_date = datetime(s_year, s_month, s_day, 0, 0, 0)
end_date = datetime(e_year, e_month, e_day, 0, 0, 0)

print("Going back in time .... ⏱")

time_start = f"{str(start_date)}Z"
end_start = f"{str(end_date)}Z"

def threadPool(values, targetFunction, maxWorkers=None):
    if maxWorkers != None:
        with concurrent.futures.ThreadPoolExecutor(max_workers=maxWorkers) as executor:
            results = list(executor.map(targetFunction, values))
    else:
        with concurrent.futures.ThreadPoolExecutor() as executor:
            results = list(executor.map(targetFunction, values))
    return results

def getMetric(metric):
    try:
        dfTimeSeries = fulcra.time_series_grouped(
                sample_rate = 60, # Value being used before 86400
                start_time = time_start,
                end_time = end_start,
                metrics=metric
        )
        value = dfTimeSeries.iloc[0, 0]
        if str(value) != "nan":
            if str(value) != "<NA>":
                if float(value) > 0.0:
                    return {
                        "Metric": metric,
                        "Name": " ".join(list(dfTimeSeries.columns)[0].split('_')),
                        "Value": value
                        }
        pass
    except Exception:
        pass

data = threadPool(metrics_, getMetric, 8)
print("On this day, ")
for each in data:
    if isinstance(each, dict):
        print("✨", each['Name'].title(), float(each['Value']).__round__(2))

display(HTML("<br>"))

calenders_id_name = {}
calenders_mine = fulcra.calendars()
for each in calenders_mine:
    calenders_id_name[each['calendar_id']] = each['calendar_name']

calendar_events = fulcra.calendar_events(start_time=start_date, end_time=end_date)
if calendar_events == []:
    print("You had no calender events for this day")
else:
    print("You had these events in your calendar on that day")
    for each_calEvent in calendar_events:
        title = each_calEvent['title']
        createdOn = each_calEvent["creation_date"].replace("Z", "").split("T")[1]
        startTime = each_calEvent['start_date'].replace("Z", "").split("T")[1]
        endTime = each_calEvent['end_date'].replace("Z", "").split("T")[1]
        eventNote = each_calEvent['notes']
        eventLocation = each_calEvent['location']
        calendarName = calenders_id_name[each_calEvent['calendar_id']]
        if each_calEvent['participants'] != None:
            totalParticipants = len(each_calEvent['participants'])
        else:
            totalParticipants = 0
        htmlCard = f'''<div style="border: 0.5px solid grey; padding: 20px; border-radius: 4px; width: 600px; margin: 20px;">
        <h4>{title}</h4>
        <h5>Start Time :</h5><p>{startTime}</p>
        <h5>End Time :</h5><p>{endTime}</p>
        <hr>
        <h5>Location :</h5><p>{eventLocation}</p>
    </div>'''
        display(HTML(htmlCard))

def mostCommonMood(data):
    events = []
    for each_event_body in data:
        events.append(each_event_body['event_body'])
    word_counts = Counter(events)
    most_common = word_counts.most_common(1)
    if most_common:
        return most_common[0][0]
    else:
        return None

display(HTML("<br>"))

filtered_events = fulcra.simple_events(
    start_time=f"{str(start_date)}.000Z", end_time=f"{str(end_date)}.000Z",
    categories=["mood"]
)
if filtered_events == []:
    print("I've searched for simple events under your category but was unable to find anything")
else:
    print(f"Your most common mood for the day was {mostCommonMood(filtered_events)}")


display(HTML("<br>"))

workouts = fulcra.apple_workouts(
    start_time = f"{str(start_date)}.000Z",
    end_time = f"{str(end_date)}.000Z"
)
timeStamp = []
METs = []
duration = []
if workouts == []:
    print("I've searched your apple workouts and found nothing")
else:
    print("I've searched for your workout and you did these workouts on that day")
    for each_wo in workouts:
        timeStamp.append(each_wo['start_date'].split('.')[0].replace('T', ' ').split(" ")[1])
        duration.append((float(each_wo['duration'])/60).__round__(2))
        METs.append(float(each_wo['extras']['HKAverageMETs'].split(' ')[0]))
    display(HTML("<br>"))
    plt.figure(figsize=(4, 4))
    plt.plot(timeStamp, METs, marker='o', linestyle='-', color='blue')
    plt.xlabel('Timestamps (HH:MM:SS)')
    plt.ylabel('AverageMETs (kcal/hr·kg)')
    plt.title('Average Metabolic Equivalent of Task per workout')
    plt.grid(True)
    plt.show()
    display(HTML("<br>"))
    plt.figure(figsize=(4, 4))
    plt.plot(timeStamp, duration, marker='o', linestyle='-', color='green')
    plt.xlabel('Timestamps (HH:MM:SS)')
    plt.ylabel('Duration of workouts')
    plt.title('Workout by duration during the day')
    plt.grid(True)
    plt.show()

display(HTML("<br>"))

locData = []
try:
    visits = fulcra.apple_location_visits(
        start_time=f'{str(start_date).replace(" ", "T")}Z',
        end_time=f'{str(end_date).replace(" ", "T")}Z'
    )
    print("You visited these locations on that day")
    display(HTML("<br>"))
    for each in visits:
        locData.append(f"{each['longitude_degrees']}:{each['latitude_degrees']}")
except Exception as e:
    print("Some error has occured while getting your location visits")

baseLong, baseLat = [float(num) for num in locData[0].split(':')]
m = folium.Map(location=[baseLong, baseLat], zoom_start=12)
for each in locData[1:]:
    long, lat = [float(num) for num in each.split(':')]
    folium.Marker(location=[long, lat], popup='Visited Locations on that day').add_to(m)
m